In [1]:
#Importando paquetes
import requests
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
import urllib.request, urllib.parse, urllib.error
from requests.packages.urllib3.util.retry import Retry
import pandas as pd

# **1. WEB SCRAPING SENAHMI**

El SENAHMI (Servicio Nacional de Meteorología e Hidrología del Perú) es la entidad encargada de proporcionar información meteorológica y climatológica en el Perú. El web scraping, una técnica que implica la extracción automatizada de datos de páginas web, se utilizará para recopilar y organizar esta valiosa información de manera eficiente. El enfoque se centrará en respetar los términos de servicio del sitio, analizar la estructura HTML para identificar los elementos relevantes y utilizar herramientas de web scraping. Este proceso permitirá obtener datos actualizados de la SENAHMI, facilitando su posterior análisis y aplicación en diversas áreas.

In [15]:
# Ingresar URL
url = 'https://www.senamhi.gob.pe/?p=pronostico-meteorologico'

# Leer el contenido HTML de la URL
html = urllib.request.urlopen(url).read()

# Crear un objeto BeautifulSoup para analizar el HTML
soup = BeautifulSoup(html, 'html.parser')

# Imprimir la sección <head> del HTML
# print(soup.head)

### **Extracción de Datos Meteorológicos por Ciudad**

In [17]:
# Encuentra todas las divisiones que contienen la información de la ciudad
city_divs = soup.find_all('div', class_='col-lg-12 p-2 m-0', style='background-color:lightblue;')

# Listas para almacenar los datos antes de crear el DataFrame
cities = []
days = []
temperature_highs = []
temperature_lows = []
descriptions = []

**Captura Eficiente de Datos Meteorológicos por Ciudad**

En este paso, iteramos sobre las divisiones de la ciudad, extraemos la información necesaria y la almacenamos en listas.

In [20]:
# Listas para almacenar la información
cities = []
days = []
temperature_highs = []
temperature_lows = []
descriptions = []

for city_div in city_divs:
    # Extrae el nombre de la ciudad
    city_name_elem = city_div.find('span', class_='nameCity')
    if city_name_elem:
        city_name = city_name_elem.text.strip()

        # Encuentra el contenedor de información del clima para la ciudad actual
        td_element = city_div.find_next('td')

        try:
            # Extrae la información del clima para cada día
            for day_info in td_element.find_all('div', class_='row m-3', recursive=False):
                day = day_info.find('div', class_='col-sm-3').text.strip()
                temperature_high = day_info.find('div', class_='col-sm-1 text-danger').strong.text.strip()
                temperature_low = day_info.find('div', class_='col-sm-1 text-primary').strong.text.strip()
                description = day_info.find('div', class_='col-sm-6').text.strip()

                # Almacena la información en las listas
                cities.append(city_name)
                days.append(day)
                temperature_highs.append(temperature_high)
                temperature_lows.append(temperature_low)
                descriptions.append(description)

        except AttributeError as e:
            # Almacena información de error en listas
            cities.append(city_name)
            days.append("Error")
            temperature_highs.append("Error")
            temperature_lows.append("Error")
            descriptions.append("Error")

# Imprime las listas después de la iteración
print("Cities:", cities)
print("Days:", days)
print("Temperature Highs:", temperature_highs)
print("Temperature Lows:", temperature_lows)
print("Descriptions:", descriptions)


Cities: ['BAGUA GRANDE - AMAZONAS', 'BAGUA GRANDE - AMAZONAS', 'BAGUA GRANDE - AMAZONAS', 'BAGUA GRANDE - AMAZONAS', 'BAGUA GRANDE - AMAZONAS', 'CHACHAPOYAS - AMAZONAS', 'CHACHAPOYAS - AMAZONAS', 'CHACHAPOYAS - AMAZONAS', 'CHACHAPOYAS - AMAZONAS', 'CHACHAPOYAS - AMAZONAS', 'CHIRIACO - AMAZONAS', 'CHIRIACO - AMAZONAS', 'CHIRIACO - AMAZONAS', 'CHIRIACO - AMAZONAS', 'CHIRIACO - AMAZONAS', 'PEDRO RUIZ - AMAZONAS', 'PEDRO RUIZ - AMAZONAS', 'PEDRO RUIZ - AMAZONAS', 'PEDRO RUIZ - AMAZONAS', 'PEDRO RUIZ - AMAZONAS', 'SANTA MARIA DE NIEVA - AMAZONAS', 'SANTA MARIA DE NIEVA - AMAZONAS', 'SANTA MARIA DE NIEVA - AMAZONAS', 'SANTA MARIA DE NIEVA - AMAZONAS', 'SANTA MARIA DE NIEVA - AMAZONAS', 'CABANA - ANCASH', 'CABANA - ANCASH', 'CABANA - ANCASH', 'CABANA - ANCASH', 'CABANA - ANCASH', 'CASMA - ANCASH', 'CASMA - ANCASH', 'CASMA - ANCASH', 'CASMA - ANCASH', 'CASMA - ANCASH', 'CHAVIN - ANCASH', 'CHAVIN - ANCASH', 'CHAVIN - ANCASH', 'CHAVIN - ANCASH', 'CHAVIN - ANCASH', 'CHIMBOTE - ANCASH', 'CHIMBOTE 

**Estructuración de Datos Meteorológicos en un DataFrame**

In [22]:
# Crea un DataFrame de pandas con los datos recopilados
df = pd.DataFrame({
    'City': cities,
    'Day': days,
    'Temperature High': temperature_highs,
    'Temperature Low': temperature_lows,
    'Description': descriptions
})

# Muestra el DataFrame
df

,City,Day,Temperature High,Temperature Low,Description
0,BAGUA GRANDE - AMAZONAS,"sábado, 13 de enero",19ºC,11ºC,Cielo nublado parcial hacia la madrugada a cie...
1,BAGUA GRANDE - AMAZONAS,"domingo, 14 de enero",19ºC,11ºC,Cielo nublado parcial a cielo nublado durante ...
2,BAGUA GRANDE - AMAZONAS,"lunes, 15 de enero",19ºC,11ºC,Cielo nublado parcial por la mañana a cielo nu...
3,BAGUA GRANDE - AMAZONAS,"martes, 16 de enero",19ºC,11ºC,Cielo nublado con viento moderado por la mañan...
4,BAGUA GRANDE - AMAZONAS,"miércoles, 17 de enero",19ºC,11ºC,Cielo nublado parcial entre cielo nublado dura...
...,...,...,...,...,...
1301,PUERTO ESPERANZA - UCAYALI,"domingo, 14 de enero",31ºC,23ºC,Cielo nublado variando a cielo nublado parcial...
1302,PUERTO ESPERANZA - UCAYALI,"lunes, 15 de enero",30ºC,23ºC,Cielo cubierto a cielo nublado durante el día ...
1303,PUERTO ESPERANZA - UCAYALI,"martes, 16 de enero",30ºC,23ºC,Cielo cubierto a cielo nublado durante el día ...
1304,PUERTO ESPERANZA - UCAYALI,"miércoles, 17 de enero",32ºC,23ºC,Cielo nublado variando a cielo nublado parcial...
